# Wrt case

In [7]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
from collections import defaultdict

# ============================================================
# USER DEFINED LISTS
# ============================================================

delta_top_list = (4, 8, 12, 14, 18)
delta_bot_list = (4, 8, 12, 16)
l_s_P_list = (-10, -7.5, -5, -2.5, 0, 2.5, 5, 7.5, 10)

# ============================================================
# REFERENCE SIMULATION FILE
# ============================================================

REFERENCE_FILE = (
    r"X:\Reddy\Caspar\V30G-205\Piston_spherical_radius\Base_with_without_micro\T60_2850rpm_350bar_100d_with_micro\output\piston\piston.txt"
)

# ============================================================
# PARSE PARAMETERS
# ============================================================

def parse_simulation_name(folder_name):
    pattern = r"lsp_(-?\d+\.?\d*)_DC(\d+)_case(\d+)"
    match = re.search(pattern, folder_name)
    if not match:
        raise ValueError
    return {
        "lsp": float(match.group(1)),
        "DC": int(match.group(2)),
        "case": int(match.group(3))
    }

# ============================================================
# COLLECT SIMULATIONS
# ============================================================

def collect_simulations(base_dir):
    sims = []
    for root, _, files in os.walk(base_dir):
        if "piston.txt" in files:
            sim_folder = os.path.basename(
                os.path.dirname(os.path.dirname(root))
            )
            try:
                params = parse_simulation_name(sim_folder)
                sims.append({
                    "file": os.path.join(root, "piston.txt"),
                    **params
                })
            except ValueError:
                pass
    return sims

# ============================================================
# REMOVE DUPLICATES
# ============================================================

def deduplicate_simulations(simulations):
    unique = {}
    simulations = sorted(simulations, key=lambda x: x["file"])
    for sim in simulations:
        key = (sim["case"], sim["DC"], sim["lsp"])
        unique[key] = sim
    return list(unique.values())

# ============================================================
# GROUP & SORT
# ============================================================

def group_simulations(simulations):
    grouped = defaultdict(lambda: defaultdict(list))
    for sim in simulations:
        grouped[sim["case"]][sim["DC"]].append(sim)

    for case in grouped:
        for DC in grouped[case]:
            grouped[case][DC].sort(
                key=lambda x: l_s_P_list.index(x["lsp"])
            )
    return grouped

# ============================================================
# COMPUTE REFERENCE AVERAGE
# ============================================================

def compute_reference_average(column):
    data = pd.read_csv(
        REFERENCE_FILE,
        delimiter="\t",
        na_values=["NaN", "Inf", "inf"]
    )
    x = data.iloc[:, 1].astype(float)
    y = data[column].astype(float)
    mask = x >= (x.max() - 1)
    return y[mask].mean()

# ============================================================
# PLOTTING FUNCTION
# ============================================================

def create_case_plots(sim_list, columns_to_plot, pdf_path):

    FIG_WIDTH = 20
    FIG_HEIGHT = 6
    POINTS_PER_DC = len(l_s_P_list)

    with PdfPages(pdf_path) as pdf:

        for col_idx, column in enumerate(columns_to_plot):

            fig, ax = plt.subplots(figsize=(FIG_WIDTH, FIG_HEIGHT))

            averages = []
            labels = []

            # ---- reference line ----
            ref_avg = compute_reference_average(column)
            ax.axhline(
                ref_avg,
                color="red",
                linestyle="--",
                linewidth=2,
                label="Reference"
            )

            # ---- main points ----
            for idx, sim in enumerate(sim_list):
                data = pd.read_csv(
                    sim["file"],
                    delimiter="\t",
                    na_values=["NaN", "Inf", "inf"]
                )

                x = data.iloc[:, 1].astype(float)
                y = data[column].astype(float)

                mask = x >= (x.max() - 1)
                avg = y[mask].mean()

                averages.append(avg)
                labels.append(f"DC{sim['DC']} | lsp={sim['lsp']}")
                ax.plot(idx, avg, 'o', markersize=8)

            # ---- vertical DC separators ----
            for i in range(1, len(delta_bot_list)):
                xpos = i * POINTS_PER_DC - 0.5
                ax.axvline(xpos, color="black", linestyle=":", alpha=0.6)

            # ---- count below reference ----
            below_count = sum(avg < ref_avg for avg in averages)

            ax.text(
                0.01, 0.98,
                f"Points below reference: {below_count}",
                transform=ax.transAxes,
                va="top",
                ha="left",
                fontsize=12,
                bbox=dict(facecolor="white", alpha=0.8, edgecolor="none")
            )

            # ---- formatting ----
            ax.set_title(
                f"{column} – Average (Last 1 Revolution)",
                fontsize=14
            )

            ax.set_xticks(range(len(labels)))
            ax.set_xticklabels(
                labels,
                rotation=45,
                ha="right",
                fontsize=10
            )

            ax.set_ylabel(column, fontsize=12)
            ax.grid(True, linestyle=":", alpha=0.7)

            if col_idx in (0, 1):
                ax.set_ylim(500, 2000)

            plt.tight_layout()
            pdf.savefig(fig)
            plt.close(fig)

# ============================================================
# MAIN
# ============================================================

if __name__ == "__main__":

    base_dir = r"X:\Reddy\Caspar\V30G-205\Piston_spherical_radius\Run4"

    downloads_dir = os.path.join(os.path.expanduser("~"), r"X:\Reddy\Caspar\V30G-205\Piston_spherical_radius\Run4")
    os.makedirs(downloads_dir, exist_ok=True)

    columns_to_plot = [
        "Mechanical_Power_Loss",
        "Total_Individual_power_loss",
        "Total_Volumetric_Power_Loss",
        "Total_Mechanical_Power_Loss",
    ]

    raw_sims = collect_simulations(base_dir)
    sims = deduplicate_simulations(raw_sims)
    grouped = group_simulations(sims)

    for case in delta_top_list:

        ordered = []
        for DC in delta_bot_list:
            ordered.extend(grouped[case][DC])

        if not ordered:
            continue

        pdf_path = os.path.join(
            downloads_dir,
            f"DeltaTop_{case}_AverageWithReference.pdf"
        )

        create_case_plots(
            ordered,
            columns_to_plot,
            pdf_path
        )

        print(f"✅ Created: {pdf_path}")

    print("🎉 All plots completed successfully.")


✅ Created: X:\Reddy\Caspar\V30G-205\Piston_spherical_radius\Run4\DeltaTop_4_AverageWithReference.pdf
✅ Created: X:\Reddy\Caspar\V30G-205\Piston_spherical_radius\Run4\DeltaTop_8_AverageWithReference.pdf
✅ Created: X:\Reddy\Caspar\V30G-205\Piston_spherical_radius\Run4\DeltaTop_12_AverageWithReference.pdf
✅ Created: X:\Reddy\Caspar\V30G-205\Piston_spherical_radius\Run4\DeltaTop_14_AverageWithReference.pdf
✅ Created: X:\Reddy\Caspar\V30G-205\Piston_spherical_radius\Run4\DeltaTop_18_AverageWithReference.pdf
🎉 All plots completed successfully.


# Wrt DC

In [8]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
from collections import defaultdict

# ============================================================
# USER DEFINED LISTS (ORDER MATTERS)
# ============================================================

delta_top_list = (4, 8, 12, 14, 18)   # case
delta_bot_list = (4, 8, 12, 16)       # DC
l_s_P_list = (-10, -7.5, -5, -2.5, 0, 2.5, 5, 7.5, 10)

# ============================================================
# REFERENCE FILE
# ============================================================

REFERENCE_FILE = (
    r"X:\Reddy\Caspar\V30G-205\Piston_spherical_radius\Base_with_without_micro\T60_2850rpm_350bar_100d_with_micro\output\piston\piston.txt"

)

# ============================================================
# PARSE SIMULATION NAME
# ============================================================

def parse_simulation_name(folder_name):
    pattern = r"lsp_(-?\d+\.?\d*)_DC(\d+)_case(\d+)"
    match = re.search(pattern, folder_name)
    if not match:
        raise ValueError
    return {
        "lsp": float(match.group(1)),
        "DC": int(match.group(2)),
        "case": int(match.group(3))
    }

# ============================================================
# COLLECT SIMULATIONS
# ============================================================

def collect_simulations(base_dir):
    sims = []
    for root, _, files in os.walk(base_dir):
        if "piston.txt" in files:
            sim_folder = os.path.basename(
                os.path.dirname(os.path.dirname(root))
            )
            try:
                params = parse_simulation_name(sim_folder)
                sims.append({
                    "file": os.path.join(root, "piston.txt"),
                    **params
                })
            except ValueError:
                pass
    return sims

# ============================================================
# REMOVE DUPLICATES
# ============================================================

def deduplicate_simulations(simulations):
    unique = {}
    simulations = sorted(simulations, key=lambda x: x["file"])
    for sim in simulations:
        key = (sim["case"], sim["DC"], sim["lsp"])
        unique[key] = sim
    return list(unique.values())

# ============================================================
# GROUP BY DC → CASE → LSP
# ============================================================

def group_by_dc(simulations):
    grouped = defaultdict(lambda: defaultdict(list))

    for sim in simulations:
        grouped[sim["DC"]][sim["case"]].append(sim)

    for DC in grouped:
        for case in grouped[DC]:
            grouped[DC][case].sort(
                key=lambda x: l_s_P_list.index(x["lsp"])
            )

    return grouped

# ============================================================
# REFERENCE AVERAGE
# ============================================================

def compute_reference_average(column):
    data = pd.read_csv(
        REFERENCE_FILE,
        delimiter="\t",
        na_values=["NaN", "Inf", "inf"]
    )
    x = data.iloc[:, 1].astype(float)
    y = data[column].astype(float)
    mask = x >= (x.max() - 1)
    return y[mask].mean()

# ============================================================
# PLOTTING FUNCTION (PER DC)
# ============================================================

def create_dc_plots(sim_list, columns_to_plot, pdf_path):

    FIG_WIDTH = 20
    FIG_HEIGHT = 6
    POINTS_PER_CASE = len(l_s_P_list)

    with PdfPages(pdf_path) as pdf:

        for col_idx, column in enumerate(columns_to_plot):

            fig, ax = plt.subplots(figsize=(FIG_WIDTH, FIG_HEIGHT))

            averages = []
            labels = []

            # ---- reference line ----
            ref_avg = compute_reference_average(column)
            ax.axhline(
                ref_avg,
                color="red",
                linestyle="--",
                linewidth=2,
                label="Reference"
            )

            # ---- plot points ----
            for idx, sim in enumerate(sim_list):

                data = pd.read_csv(
                    sim["file"],
                    delimiter="\t",
                    na_values=["NaN", "Inf", "inf"]
                )

                x = data.iloc[:, 1].astype(float)
                y = data[column].astype(float)

                mask = x >= (x.max() - 1)
                avg = y[mask].mean()

                averages.append(avg)
                labels.append(f"case{sim['case']} | lsp={sim['lsp']}")

                ax.plot(idx, avg, 'o', markersize=8)

            # ---- vertical separators between CASES ----
            for i in range(1, len(delta_top_list)):
                xpos = i * POINTS_PER_CASE - 0.5
                ax.axvline(xpos, color="black", linestyle=":", alpha=0.6)

            # ---- count below reference ----
            below_count = sum(avg < ref_avg for avg in averages)

            ax.text(
                0.01, 0.98,
                f"Points below reference: {below_count}",
                transform=ax.transAxes,
                va="top",
                ha="left",
                fontsize=12,
                bbox=dict(facecolor="white", alpha=0.8, edgecolor="none")
            )

            # ---- formatting ----
            ax.set_title(
                f"{column} – Average (Last 1 Revolution)",
                fontsize=14
            )

            ax.set_xticks(range(len(labels)))
            ax.set_xticklabels(
                labels,
                rotation=45,
                ha="right",
                fontsize=10
            )

            ax.set_ylabel(column, fontsize=12)
            ax.grid(True, linestyle=":", alpha=0.7)

            if col_idx in (0, 1):
                ax.set_ylim(500, 2000)

            plt.tight_layout()
            pdf.savefig(fig)
            plt.close(fig)

# ============================================================
# MAIN
# ============================================================

if __name__ == "__main__":

    base_dir = r"X:\Reddy\Caspar\V30G-205\Piston_spherical_radius\Run4"

    downloads_dir = os.path.join(os.path.expanduser("~"), r"X:\Reddy\Caspar\V30G-205\Piston_spherical_radius\Run4")
    os.makedirs(downloads_dir, exist_ok=True)

    columns_to_plot = [
        "Mechanical_Power_Loss",
        "Total_Individual_power_loss",
        "Total_Volumetric_Power_Loss",
        "Total_Mechanical_Power_Loss",
    ]

    raw_sims = collect_simulations(base_dir)
    sims = deduplicate_simulations(raw_sims)
    grouped = group_by_dc(sims)

    for DC in delta_bot_list:

        ordered = []
        for case in delta_top_list:
            ordered.extend(grouped[DC][case])

        if not ordered:
            continue

        pdf_path = os.path.join(
            downloads_dir,
            f"DC_{DC}_CaseLSP_Comparison.pdf"
        )

        create_dc_plots(
            ordered,
            columns_to_plot,
            pdf_path
        )

        print(f"✅ Created: {pdf_path}")

    print("🎉 All DC-based plots completed successfully.")


✅ Created: X:\Reddy\Caspar\V30G-205\Piston_spherical_radius\Run4\DC_4_CaseLSP_Comparison.pdf
✅ Created: X:\Reddy\Caspar\V30G-205\Piston_spherical_radius\Run4\DC_8_CaseLSP_Comparison.pdf
✅ Created: X:\Reddy\Caspar\V30G-205\Piston_spherical_radius\Run4\DC_12_CaseLSP_Comparison.pdf
✅ Created: X:\Reddy\Caspar\V30G-205\Piston_spherical_radius\Run4\DC_16_CaseLSP_Comparison.pdf
🎉 All DC-based plots completed successfully.
